# Import packages

In [15]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np

# Downloading data 

In [3]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareclimate_data/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]

In [5]:
%%time
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 3.02 s, sys: 3.4 s, total: 6.41 s
Wall time: 47 s


## Extract contents of zipped file

In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(f'{output_directory}/data')

CPU times: user 16.7 s, sys: 2.65 s, total: 19.3 s
Wall time: 19.5 s


## Remove unused file

In [7]:
unused_file = os.path.join(
    output_directory,
    "data/observed_daily_rainfall_SYD.csv")
if os.path.exists(unused_file):
    os.remove(unused_file)

# Combine data CSVs

In [8]:
%%time
files = glob.glob('figshareclimate_data/data/*.csv')
df = pd.concat(
    (pd.read_csv(file, index_col=0, parse_dates=['time'])
     .assign(model=re.findall(r'[^\/]+(?=_daily_rainfall_NSW\.)', file)[0])
     for file in files)
)
df.to_csv("figshareclimate_data/combined_data.csv")

CPU times: user 8min 5s, sys: 17.2 s, total: 8min 22s
Wall time: 8min 32s


In [9]:
print(df.shape)

(62467843, 6)


In [10]:
df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 62467843 entries, 1889-01-01 12:00:00 to 2014-12-31 12:00:00
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   lat_min        float64
 1   lat_max        float64
 2   lon_min        float64
 3   lon_max        float64
 4   rain (mm/day)  float64
 5   model          object 
dtypes: float64(5), object(1)
memory usage: 3.3+ GB


In [12]:
df.dtypes

lat_min          float64
lat_max          float64
lon_min          float64
lon_max          float64
rain (mm/day)    float64
model             object
dtype: object

## Combine data csv on different machines

| Team Member            | Operating System | RAM | Processor | Is SSD | Time taken |
|:----------------------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Kingslin Lv            | Windows          | 16GB|    i7      | Yes     | 15min 15s   |
| Sufang Tan             |     |           |        |            |
| Amir Abbas Shojakhani  |Mac OS     |16GB           |i7       |Yes           |08min 50s

# EDA-Python

## 1st Approach - Changing dtype of the data

### Value counts with base data types

In [14]:
%%time
df.value_counts()

CPU times: user 1min 29s, sys: 15.7 s, total: 1min 44s
Wall time: 1min 47s


lat_min  lat_max  lon_min   lon_max   rain (mm/day)  model        
-30.625  -29.375  141.5625  143.4375  0.000000       ACCESS-ESM1-5    15271
-31.875  -30.625  141.5625  143.4375  0.000000       ACCESS-ESM1-5    13850
-30.625  -29.375  143.4375  145.3125  0.000000       ACCESS-ESM1-5    13615
-31.875  -30.625  143.4375  145.3125  0.000000       ACCESS-ESM1-5    12638
-33.125  -31.875  141.5625  143.4375  0.000000       ACCESS-ESM1-5    12112
                                                                      ...  
-34.000  -33.000  148.7500  150.0000  0.000702       GFDL-CM4             1
                                      0.000704       GFDL-CM4             1
                                                     GFDL-CM4             1
                                                     GFDL-CM4             1
-29.900  -29.100  152.7250  153.5250  199.089043     FGOALS-f3-L          1
Length: 55839634, dtype: int64

Value counts with dtype conversion from float64 to float32

In [21]:
df_conv = df.copy()
df_conv['lat_min'] = np.float32(df_conv["lat_min"])
df_conv['lat_max'] = np.float32(df_conv["lat_max"])
df_conv['lon_min'] = np.float32(df_conv["lon_min"])
df_conv['lon_max'] = np.float32(df_conv["lon_max"])

In [22]:
%%time
df.value_counts()

CPU times: user 1min 29s, sys: 17.9 s, total: 1min 47s
Wall time: 1min 49s


lat_min  lat_max  lon_min     lon_max     rain (mm/day)  model        
-30.625  -29.375  141.562500  143.437500  0.000000       ACCESS-ESM1-5    15271
-31.875  -30.625  141.562500  143.437500  0.000000       ACCESS-ESM1-5    13850
-30.625  -29.375  143.437500  145.312500  0.000000       ACCESS-ESM1-5    13615
-31.875  -30.625  143.437500  145.312500  0.000000       ACCESS-ESM1-5    12638
-33.125  -31.875  141.562500  143.437500  0.000000       ACCESS-ESM1-5    12112
                                                                          ...  
-34.000  -33.000  148.750000  150.000000  0.000702       GFDL-CM4             1
                                          0.000704       GFDL-CM4             1
                                                         GFDL-CM4             1
                                                         GFDL-CM4             1
-29.900  -29.100  152.725006  153.524994  199.089043     FGOALS-f3-L          1
Length: 55839634, dtype: int64

## 2nd Approach - Load just columns we want

In [31]:
df_reduced = df[['lat_min', 'lat_max', 'lon_min', 'lon_max']]
df_reduced = df_reduced.reset_index().drop('time', axis=1)
df_reduced.head()

,lat_min,lat_max,lon_min,lon_max
0,-35.439865,-33.574619,141.5625,143.4375
1,-35.439865,-33.574619,141.5625,143.4375
2,-35.439865,-33.574619,141.5625,143.4375
3,-35.439865,-33.574619,141.5625,143.4375
4,-35.439865,-33.574619,141.5625,143.4375


In [33]:
%%time
df.value_counts()

CPU times: user 1min 29s, sys: 19.1 s, total: 1min 48s
Wall time: 1min 53s


lat_min  lat_max  lon_min     lon_max     rain (mm/day)  model        
-30.625  -29.375  141.562500  143.437500  0.000000       ACCESS-ESM1-5    15271
-31.875  -30.625  141.562500  143.437500  0.000000       ACCESS-ESM1-5    13850
-30.625  -29.375  143.437500  145.312500  0.000000       ACCESS-ESM1-5    13615
-31.875  -30.625  143.437500  145.312500  0.000000       ACCESS-ESM1-5    12638
-33.125  -31.875  141.562500  143.437500  0.000000       ACCESS-ESM1-5    12112
                                                                          ...  
-34.000  -33.000  148.750000  150.000000  0.000702       GFDL-CM4             1
                                          0.000704       GFDL-CM4             1
                                                         GFDL-CM4             1
                                                         GFDL-CM4             1
-29.900  -29.100  152.725006  153.524994  199.089043     FGOALS-f3-L          1
Length: 55839634, dtype: int64

| Team Member            | Operating System | RAM | Processor | Is SSD | Time taken for EDA on base dataframe |Time taken for EDA after changing dtypes |Time taken for EDA after reducing features|
|:----------------------:|:----------------:|:---:|:---------:|:------:|:----------:|:----------:|:----------:|
| Kingslin Lv            | Windows          | 16GB|    i7      | Yes     |   |   |   |
| Sufang Tan             |     |           |        |            |            |            |
| Amir Abbas Shojakhani  |Mac OS     |16GB           |i7       |Yes           |01min 47s|01min 49s|01min 53s|